#### Documentação:
https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/
https://python.langchain.com/docs/integrations/memory/
https://python.langchain.com/v0.2/docs/how_to/message_history/

#### Definição

Passar o estado da conversa para dentro e para fora de uma cadeia é necessário para se construir um chatbot. Isso é o que chamamos de histórico de conversa, ou seja, quando você armazenas as trocas de mensagens de uma sessão de conversa e enviar para o modelo para que ele possa entender o contexto conversado até aquele ponto da conversa.

Para isso, hoje a documentação do LangChain sugere que usemos o conceito de estados (`state`) do LagnGraph para controlar o histórico de mensagens, mas eles fornecem uma opção com classes puras e que não serão descontinuadas do próprio LangChain: `RunnableWithMessageHistory`, que nos permite adicionar histórico de mensagens a certos tipos de cadeias. 

A classe `RunnableWithMessageHistory` envolve outro `Runnable` (como por exemplo nossa chain) e gerencia o histórico de mensagens de chat para ele. Especificamente, ela carrega mensagens anteriores na conversa **ANTES** de passá-la para o `Runnable`, e salva a resposta gerada como uma mensagem **DEPOIS** de chamar o `runnable`.  Além disso, esta classe permite isolar uma conversa da outra usando um `session_id`  sendo passado na configuração ao chamar o runnable, e usa isso para procurar o histórico de conversas relevante.

### Como Armazenar e Carregar Mensagens em Fluxos Conversacionais com LangChain

#### Passo 1: Compreendendo os Conceitos Fundamentais

#### O que é `session_id`?

O `session_id` é um identificador exclusivo para uma sessão ou thread de conversa. Ele é usado para associar mensagens de entrada e saída a uma conversa específica, possibilitando o gerenciamento de múltiplos diálogos de forma independente.

### O que é `BaseChatMessageHistory`?

A classe `BaseChatMessageHistory` é usada para salvar e carregar objetos de mensagens. Ela é essencial para gerenciar o histórico das conversas, sendo invocada por componentes como o `RunnableWithMessageHistory`. Geralmente, essa classe é inicializada com um `session_id`.

#### Passo 2: Criando uma Função `get_session_history`

Para implementar o gerenciamento de histórico de mensagens, precisamos criar uma função chamada `get_session_history`. Essa função será responsável por retornar um objeto de histórico de mensagens associado a um `session_id`.

Neste exemplo, utilizaremos o **SQLite** para armazenar o histórico de mensagens de forma simples e prática.

#### Código de Implementação

Antes de começar, certifique-se de que o banco de dados local esteja limpo, excluindo-o do diretório, se existir:

Agora, implemente a função `get_session_history`:

#### Executar imagem do docker do Redis

```bash
docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```


In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from helpers.llm import initialize_llm, logger, pretty_print

llm, _, _ = initialize_llm()

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Função para retornar o histórico de mensagens com base no session_id
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

Aqui está o que acontece no código acima:

1. **Importação do `SQLChatMessageHistory`**: Este é um dos adaptadores disponíveis para gerenciar o histórico de mensagens, usando um banco SQLite como backend.

2. **Função `get_session_history`**: Retorna um objeto `SQLChatMessageHistory` associado a um `session_id` específico. O banco de dados é definido pelo URI `sqlite:///memory.db`.

### Passo 3: Utilizando o Histórico em Seus Fluxos

Depois de implementar a função `get_session_history`, você pode integrá-la com o `RunnableWithMessageHistory`. Este componente será responsável por conectar seu modelo de linguagem com o histórico de mensagens, permitindo que o contexto seja mantido durante a conversa.

**Usando Dicionários como Entrada e Saída**

Se o seu fluxo envolve prompts com variáveis dinâmicas, você pode usar um **dicionário** para entrada e saída. Nesse caso, é necessário configurar:

1. **`input_messages_key`**: Chave no dicionário que contém a mensagem de entrada.
2. **`history_messages_key`**: Chave que será usada para armazenar mensagens históricas.

In [ ]:

# Configurando o prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente que responde em {language}. Use no máximo 20 palavras."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# Conectando o prompt ao modelo
runnable = prompt | llm

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# Invocando com variáveis dinâmicas
response = runnable_with_history.invoke(
    {"language": "portugue", "input": "hi im bob!"},
    config={"configurable": {"session_id": "2"}},
)
pretty_print(response)

In [ ]:
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "My favorite fruit is apple. What color is it?"},
    config={"configurable": {"session_id": "2"}},
)
pretty_print(response)

In [ ]:
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "What is my favorite fruit?"},
    config={"configurable": {"session_id": "2"}},
)
pretty_print(response)

In [ ]:
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "I have a yellow car, my car is broken because I have a flat tire."},
    config={"configurable": {"session_id": "2"}},
)
pretty_print(response)

In [ ]:
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "What happened to my car being broken?"},
    config={"configurable": {"session_id": "2"}},
)
pretty_print(response)

### Configuração Avançada com Chaves Personalizadas

Você pode adicionar camadas de personalização no rastreamento de histórico, por exemplo, usando identificadores como `user_id` e `conversation_id`. Isso permite gerenciar múltiplos usuários em um único sistema.


In [ ]:
from langchain_core.runnables import ConfigurableFieldSpec
import uuid

# Generate a random UUID
random_user_id = uuid.uuid4().hex

print(f"Random User ID: {random_user_id}")

def get_session_history(user_id: str, conversation_id: str):
    return SQLChatMessageHistory(f"{user_id}--{conversation_id}", "sqlite:///memory.db")

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Identificador único para o usuário.",
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Identificador único para a conversa.",
        ),
    ],
)

# Invocando com identificadores personalizados
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "Hello, im Grassato!"},
    config={"configurable": {"user_id": random_user_id, "conversation_id": "1"}},
)

response = runnable_with_history.invoke(
    {"language": "portugues", "input": "My favorite fruit is apple. What color is it?"},
    config={"configurable": {"user_id": random_user_id, "conversation_id": "1"}},
)

response = runnable_with_history.invoke(
    {"language": "portugues", "input": "I have a yellow car, my car is broken because I have a flat tire."},
    config={"configurable": {"user_id": random_user_id, "conversation_id": "1"}},
)
pretty_print(response)
 

In [ ]:
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "What is my favorite fruit?"},
    config={"configurable": {"user_id": random_user_id, "conversation_id": "1"}},
)
pretty_print(response)

In [ ]:
response = runnable_with_history.invoke(
    {"language": "portugues", "input": "What happened to my car being broken?"},
    config={"configurable": {"user_id": random_user_id, "conversation_id": "1"}},
)
pretty_print(response)

##### Observação Importante

Ao usar o SQLite para armazenar mensagens, tenha em mente que este método é mais adequado para testes e pequenos projetos. Para aplicações em produção, é recomendável usar sistemas de banco de dados mais robustos, como Redis ou PostgreSQL.

### Outros tipos de gerenciadores de mensagem

Há outras formas de fazer o armazenamento e gerenciamento de mensagens de históricos, convido você a explorar a [documentação de integrações do LangChain](https://python.langchain.com/v0.2/docs/integrations/memory/).

#### Controlando o tamanho do contexto de histórico

LLMs e modelos de bate-papo têm janelas de contexto limitadas e, mesmo que você não esteja atingindo os limites diretamente, você pode querer limitar a quantidade de distração com a qual o modelo tem que lidar. Uma solução é 'cortar' as mensagens históricas antes de passá-las para o modelo. Para que você faça esse corte, você precisa usar uma função do LangChain que faz este trabalho para você.

Neste exemplo, `trim_messages` gerencia o total de mensagens utilizando a estratégia `last`, ou seja, pegando sempre as últimas mensagens trocadas entre o usuário e o chatbot em um tamanho de 2 ao considerar `token_counter=len`.

```python
from operator import itemgetter

from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough

trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt
    | chat
)

chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
```

#### Prática

Agora vamos desenvolver um chatbot com gerenciamento de histórico. Para isso, vamos resgatar nosso assistente de atendimento odontológico que criamos em aulas anteriores. Com algumas modificações e para simplificar o processo, vou tirar a chain de roteamento inicial e deixar apenas o assistente automático de atendimento. Com isso podemos inserir um gerenciador de histórico usando o SQLite.

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.chat_history import BaseChatMessageHistory

from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
#from langchain_redis import RedisChatMessageHistory
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

## Criando o gestor de memória (histórico)
# Função para retornar o histórico de mensagens com base no session_id

def get_session_history(session_id) -> BaseChatMessageHistory:
    #return RedisChatMessageHistory(session_id, redis_url=REDIS_URL)
    return SQLChatMessageHistory(session_id, connection="sqlite:///memory.db")

 

# --------------------------------------------------------------------------------

# Definindo o prompt de chatbot que tira duvidas do usuário:
sys_chatbot_prompt = """ Você é um assistente de uma clinica odontológica e tem como objetivo responder à perguntas dos clientes. A seguir você 
encontra a FAQ do nosso site, use essas informações para realizar o atendimento e tirar dúvidas. Caso você desconheça alguma
informação, não invente. Seja sempre amigável e esteja disposto a ajudar!  

**FAQ - Clínica Odontológica**  
1. **Quais serviços a clínica oferece?**    
   Oferecemos tratamentos como limpeza dental, clareamento, ortodontia, implantes, próteses, tratamento de canal e estética dental.  
2. **A clínica aceita convênios?**    
   Sim, trabalhamos com os principais convênios odontológicos. Consulte nossa equipe para verificar se aceitamos o seu.  
3. **Como agendar uma consulta?**    
   Você pode agendar sua consulta pelo telefone, WhatsApp ou diretamente em nosso site.  
4. **Quanto tempo dura uma consulta?**    
   Depende do procedimento, mas consultas de rotina geralmente duram entre 30 e 60 minutos.  
5. **Vocês atendem emergências?**    
   Sim, oferecemos atendimento emergencial para dores agudas, traumas ou casos de urgência.  
6. **É possível parcelar tratamentos?**    
   Sim, oferecemos opções de parcelamento. Entre em contato para conhecer os detalhes.  
7. **Crianças podem ser atendidas na clínica?**    
   Sim, contamos com profissionais especializados em odontopediatria para cuidar dos sorrisos dos pequenos.  
8. **O clareamento dental é seguro?**    
   Sim, nossos tratamentos de clareamento são realizados com técnicas e produtos seguros, supervisionados por especialistas.  
Se tiver mais dúvidas, entre em contato conosco! 😊  
"""

# Incluindo a posição do MessagesPlaceholder onde será incluindo a lista de mensagens de histórico.
prompt_template_chatbot = ChatPromptTemplate.from_messages([
    ("system", sys_chatbot_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("human", "Dúvida do usuário: {input}"),
]
)

chain_chatbot = prompt_template_chatbot | llm | StrOutputParser()

# --------------------------------------------------------------------------------
## Encapsulando nossa chain com a classe de gestão de mensagens de histórico
runnable_with_history = RunnableWithMessageHistory(
    chain_chatbot,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)
# --------------------------------------------------------------------------------

# Executando nossa chain principal.
result = runnable_with_history.invoke(
    {"input": "Olá, Sou Diego, tudo bem?"},
    config={"configurable": {"session_id": "1"}},
)
print(result)

result = runnable_with_history.invoke(
    {"input": "O clareamento dental é seguro?"},
    config={"configurable": {"session_id": "1"}},
)
print(result)

result = runnable_with_history.invoke(
    {"input": "Eu precisaria parcelar, como funciona esse processo? Posso fazer?"},
    config={"configurable": {"session_id": "1"}},
)

# --------------------------------------------------------------------------------
# Imprimindo a saida.

print(result)
print("---------------")


# Executando nossa chain principal.
result = runnable_with_history.invoke(
    {"input": "Quem sou eu? O que eu estou procurando? Como pode me ajudar?"},
    config={"configurable": {"session_id": "1"}},
)
 

# Imprimindo a saida.
print("---------------")
print(result)
print("---------------")

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
 
# --------------------------------------------------------------------------------

## Criando o gestor de memória (histórico)
# Função para retornar o histórico de mensagens com base no session_id

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, connection="sqlite:///memory.db")


# --------------------------------------------------------------------------------

# Definindo o prompt de chatbot que tira duvidas do usuário:

sys_chatbot_prompt = """ Você é um assistente de uma clinica odontológica e tem como objetivo responder à perguntas dos clientes. A seguir você  
encontra a FAQ do nosso site, use essas informações para realizar o atendimento e tirar dúvidas. Caso você desconheça alguma 
informação, não invente. Seja sempre amigável e esteja disposto a ajudar!  

**FAQ - Clínica Odontológica**  
1. **Quais serviços a clínica oferece?**    
   Oferecemos tratamentos como limpeza dental, clareamento, ortodontia, implantes, próteses, tratamento de canal e estética dental.  
2. **A clínica aceita convênios?**    
   Sim, trabalhamos com os principais convênios odontológicos. Consulte nossa equipe para verificar se aceitamos o seu.  
3. **Como agendar uma consulta?**    
   Você pode agendar sua consulta pelo telefone, WhatsApp ou diretamente em nosso site.  
4. **Quanto tempo dura uma consulta?**    
   Depende do procedimento, mas consultas de rotina geralmente duram entre 30 e 60 minutos.  
5. **Vocês atendem emergências?**    
   Sim, oferecemos atendimento emergencial para dores agudas, traumas ou casos de urgência.  
6. **É possível parcelar tratamentos?**    
   Sim, oferecemos opções de parcelamento. Entre em contato para conhecer os detalhes.  
7. **Crianças podem ser atendidas na clínica?**    
   Sim, contamos com profissionais especializados em odontopediatria para cuidar dos sorrisos dos pequenos.  
8. **O clareamento dental é seguro?**    
   Sim, nossos tratamentos de clareamento são realizados com técnicas e produtos seguros, supervisionados por especialistas.  
Se tiver mais dúvidas, entre em contato conosco! 😊  
"""

prompt_template_chatbot = ChatPromptTemplate.from_messages([
    ("system", sys_chatbot_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("human", "Dúvida do usuário: {input}"),
]
)

chain_chatbot = prompt_template_chatbot | llm | StrOutputParser()

# --------------------------------------------------------------------------------
## Encapsulando nossa chain com a classe de gestão de mensagens de histórico
# Criando a função que corta o histórico de mensagem para 2 ultimas mensagens trocadas:
from operator import itemgetter

from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough

trimmer = trim_messages(strategy="first", max_tokens=2, token_counter=len)

chain_with_trimming = (
    RunnablePassthrough.assign(history=itemgetter("history") | trimmer)
    | chain_chatbot
)

runnable_with_history = RunnableWithMessageHistory(
    chain_with_trimming,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)
# --------------------------------------------------------------------------------

# Executando nossa chain principal.
result = runnable_with_history.invoke(
    {"input": "Olá sou Diego, sou Software Enginier da Avanade"},
    config={"configurable": {"session_id": "1"}},
)


# Executando nossa chain principal.
result = runnable_with_history.invoke(
    {"input": "Hoje eu resido de São José do Rio preto, vocês atendem?"},
    config={"configurable": {"session_id": "1"}},
)

# Executando nossa chain principal.
result = runnable_with_history.invoke(
    {"input": "Olá tudo bem? Qual meu nome? Onde resido? Qual minha profissão? Qual a primeira pergunta que eu realizei?"},
    config={"configurable": {"session_id": "1"}},
)

# --------------------------------------------------------------------------------
# Imprimindo a saida.
print("---------------")
print(result)
print("---------------")